In [130]:
import maboss
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os

In [148]:
model_file = "../maboss/models/model_two.bnd"
cfg_file = "../maboss/models/model_two_base.cfg"

# WT
model = maboss.load(model_file, cfg_file)
model.get_initial_state()

{'DSP': [0, 1], 'TJP1': [0, 1], 'JCAD': [0, 1], 'RhoA': [0, 1], 'RhoC': [0, 1]}

In [149]:
# DSP single KO
m = model.copy()
m.mutate("DSP", "OFF")

# DSP + JCAD double KO
m2 = model.copy()
m2.mutate("DSP", "OFF")
m2.mutate("JCAD", "OFF")

# JCAD single KO
m3 = model.copy()
m3.mutate("JCAD", "OFF")

# TJP1 single KO
m4 = model.copy()
m4.mutate("TJP1", "OFF")


# TJP1 + JCAD double KO
m5 = model.copy()
m5.mutate("TJP1", "OFF")
m5.mutate("JCAD", "OFF")

In [158]:
res = m5.run()
prob = res.get_nodes_probtraj()
nodes_df = prob.copy()
nodes_df["delta"] = nodes_df["RhoC"] - nodes_df["RhoA"]
nodes_df

,DSP,TJP1,JCAD,RhoA,RhoC,delta
0.0,0.499901,0.0,0.0,0.508179,0.481669,-0.026510
0.1,0.499900,0.0,0.0,0.532465,0.460958,-0.071507
0.2,0.499900,0.0,0.0,0.554002,0.447636,-0.106366
0.3,0.499900,0.0,0.0,0.571084,0.439541,-0.131543
0.4,0.499900,0.0,0.0,0.585297,0.434235,-0.151062
0.5,0.499899,0.0,0.0,0.597507,0.428502,-0.169005
0.6,0.499900,0.0,0.0,0.604032,0.424646,-0.179386
0.7,0.499901,0.0,0.0,0.613708,0.423187,-0.190521
0.8,0.499899,0.0,0.0,0.619746,0.419226,-0.200520
0.9,0.499900,0.0,0.0,0.624904,0.418865,-0.206039


## Threshold (eps) Method

Pros:

Very clear biological interpretation: hyper only when RhoC is significantly higher than RhoA.

Works well to capture knockout phenotypes in line with biology.

Easy to adjust eps for sensitivity.

Cons:

Binary: no information on magnitude of dominance, loses fine probabilistic detail.

Requires choosing eps, which may need calibration.

Use case: Best for categorical phenotypes where you want clear assignment (e.g., WT = Normal, DSP KO = Hyper).

In [122]:
delta = nodes_df["delta"]

eps = 0.3
hyper   = (delta > eps).astype(float)  # RhoC significantly higher than RhoA
failed  = (delta < -eps).astype(float) # RhoA significantly higher than RhoC
normal  = ((abs(delta) <= eps)).astype(float)

pd.DataFrame({
        "Failed": failed,
        "Hyper": hyper,
        "Normal": normal
}, index=nodes_df.index)

,Failed,Hyper,Normal
0.0,0.0,0.0,1.0
0.1,0.0,0.0,1.0
0.2,0.0,0.0,1.0
0.3,0.0,0.0,1.0
0.4,0.0,0.0,1.0
0.5,0.0,0.0,1.0
0.6,0.0,0.0,1.0
0.7,0.0,1.0,0.0
0.8,0.0,1.0,0.0
0.9,0.0,1.0,0.0


## Fractional Dominance Method

Pros:

Probabilistic, continuous, captures the relative contributions of RhoA vs RhoC.

Works well if both nodes have low-level fluctuations — gives smooth phenotypes.

Automatically bounded [0,1].

Cons:

Small numbers can cause numerical instability (division by tiny sums).

Doesn’t explicitly threshold what “dominance” means; even tiny differences contribute to hyper/failed.

Use case: Good for time-course or probabilistic weighting, not strict binary phenotype.

In [123]:
A = nodes_df["RhoA"]
C = nodes_df["RhoC"]
delta = C - A

hyper = (C / (A + C)).clip(0, 1)
failed  = (A / (A + C)).clip(0, 1)
normal  = 1 - hyper - failed

pd.DataFrame({
        "Failed": failed,
        "Hyper": hyper,
        "Normal": normal
}, index=nodes_df.index).tail()

,Failed,Hyper,Normal
3.5,0.227395,0.772605,8.326673e-17
3.6,0.226936,0.773064,5.551115e-17
3.7,0.226157,0.773843,0.000000e+00
3.8,0.225420,0.774580,-5.551115e-17
3.9,0.223817,0.776183,-2.775558e-17


## Combined

In [159]:
A = nodes_df["RhoA"]
C = nodes_df["RhoC"]
delta = C - A
eps = 0.25

# Binary classification
hyper_bin   = (delta > eps).astype(float)
failed_bin  = (delta < -eps).astype(float)
normal_bin  = ((abs(delta) <= eps)).astype(float)

pheno_df = pd.DataFrame({
        "Failed": failed_bin,
        "Hyper": hyper_bin,
        "Normal": normal_bin
}, index=nodes_df.index)
pheno_df


,Failed,Hyper,Normal
0.0,0.0,0.0,1.0
0.1,0.0,0.0,1.0
0.2,0.0,0.0,1.0
0.3,0.0,0.0,1.0
0.4,0.0,0.0,1.0
0.5,0.0,0.0,1.0
0.6,0.0,0.0,1.0
0.7,0.0,0.0,1.0
0.8,0.0,0.0,1.0
0.9,0.0,0.0,1.0


In [143]:
# Probabilistic weighting
total = A + C + 1e-9
hyper_prob = (C / total) * hyper_bin
failed_prob = (A / total) * failed_bin
normal_prob = normal_bin

prob_df = pd.DataFrame({
        "Failed": failed_prob,
        "Hyper": hyper_prob,
        "Normal": normal_prob
}, index=nodes_df.index)
prob_df

,Failed,Hyper,Normal
0.0,0.0,0.0,1.0
0.1,0.0,0.0,1.0
0.2,0.0,0.0,1.0
0.3,0.0,0.0,1.0
0.4,0.0,0.0,1.0
0.5,0.0,0.0,1.0
0.6,0.0,0.0,1.0
0.7,0.0,0.0,1.0
0.8,0.0,0.0,1.0
0.9,0.0,0.0,1.0


,<nil>,DSP,DSP -- JCAD,DSP -- JCAD -- RhoA,DSP -- JCAD -- RhoC,DSP -- JCAD -- RhoC -- RhoA,DSP -- RhoA,DSP -- RhoC,DSP -- RhoC -- RhoA,DSP -- TJP1,DSP -- TJP1 -- JCAD,DSP -- TJP1 -- JCAD -- RhoA,DSP -- TJP1 -- JCAD -- RhoC,DSP -- TJP1 -- JCAD -- RhoC -- RhoA,DSP -- TJP1 -- RhoA,DSP -- TJP1 -- RhoC,DSP -- TJP1 -- RhoC -- RhoA,JCAD,JCAD -- RhoA,JCAD -- RhoC,JCAD -- RhoC -- RhoA,RhoA,RhoC,RhoC -- RhoA,TJP1,TJP1 -- JCAD,TJP1 -- JCAD -- RhoA,TJP1 -- JCAD -- RhoC,TJP1 -- JCAD -- RhoC -- RhoA,TJP1 -- RhoA,TJP1 -- RhoC,TJP1 -- RhoC -- RhoA
3.5,0.015748,0.010566,0.003547,0.076915,0.007949,0.035789,0.057341,0.029609,0.028283,0.008223,0.000547,0.018417,0.016649,0.088286,0.040519,0.038080,0.039177,0.016038,0.044895,0.042696,0.022171,0.040312,0.046365,0.021775,0.009556,0.002701,0.006588,0.080228,0.036583,0.028633,0.057131,0.028680
3.6,0.016334,0.010582,0.003719,0.075788,0.007012,0.037681,0.056003,0.029707,0.029508,0.007844,0.000628,0.018177,0.016590,0.088505,0.041081,0.038128,0.038947,0.017337,0.043710,0.042151,0.022603,0.039761,0.045997,0.022108,0.010111,0.002996,0.007182,0.079251,0.036672,0.027501,0.056989,0.029399
3.7,0.016711,0.011085,0.003507,0.075189,0.008114,0.037390,0.054686,0.029954,0.030075,0.007724,0.000793,0.017767,0.018216,0.087125,0.041310,0.038761,0.038206,0.017664,0.042047,0.042262,0.023827,0.039640,0.045774,0.022076,0.009733,0.002501,0.006581,0.078919,0.038099,0.026443,0.056717,0.031107
3.8,0.017092,0.011847,0.003260,0.074747,0.008472,0.037721,0.053410,0.030590,0.029953,0.007926,0.000453,0.018703,0.018669,0.086074,0.039925,0.037804,0.040345,0.018064,0.041677,0.041883,0.024175,0.039848,0.045931,0.021328,0.009334,0.002804,0.006050,0.078208,0.039038,0.026388,0.056622,0.031655
3.9,0.015938,0.012523,0.003193,0.073853,0.007851,0.039303,0.052699,0.030652,0.029926,0.007959,0.000987,0.018854,0.017821,0.086239,0.040534,0.037674,0.039833,0.018074,0.041522,0.041244,0.024960,0.041130,0.046427,0.020706,0.009580,0.002600,0.005667,0.078623,0.039210,0.026349,0.056722,0.031348
